In [4]:
### File to DB
# 1. CSV to table 
#   CSV파일은 콤마를 기준으로 열단위 자료를 작성하는 파일로 데이터베이스의 
# 테이블 구조와 호환성이 높다. 
# 
#  1단계 : 테이블 생성 -> 레코드 추가
# #

import pandas as pd
import pymysql

# 1. CSV파일을 읽기
bmi = pd.read_csv("data/bmi.csv")
bmi.info()
'''
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
'''

# 각 컬럼 추출
height = bmi['height']
weight = bmi['weight']
label = bmi['label']

# DB연동 환경 정보
config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True    
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    # 테이블 확인(show tables)
    cursor.execute("show tables")
    tables = cursor.fetchall()
    
    # 스위칭 기법
    sw = False
    for table in tables:
        print(table,type(table))
        if table[0] == 'bmi_tab':
            sw = True   # table 존재하는 경우
    
    # table 실행
    if not sw:
        print("테이블 없음")  # 테이블 생성
        sql = """create table bmi_tab(
            height int not null,
            weight int not null,
            label varchar(15) not null
        )""" 
        cursor.execute(sql)
    
    # 레코드 조회
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall()
    
    if rows:   ## 레코드가 있는 경우
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}")
        print("전체 레코드 수 : ",len(rows))
    else:      ## 레코드가 없는 경우 
        print("100 레코드 추가")
        for i in range(100):
            h = height[i]
            w = weight[i]
            lab = label[i]
            cursor.execute(f"insert into bmi_tab values({h},{w},'{lab}')")
            conn.commit()                    

except Exception as e:
    print("DB 연동 에러 : ",e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB
('goods',) <class 'tuple'>
테이블 없음
100 레코드 추가


In [17]:
### score.csv 파일을 DB 저장하세요... 
### table명 score_tab으로 생성하세요....

#import
import pandas as pd
import pymysql

# score.csv 파일 읽기
score = pd.read_csv('data/score.csv')
#print(score)
#print(score.info())
#print(len(score))
# 컬럼 추출
no = score['no']
kor = score.kor
eng = score.eng
mat = score.mat
dept = score.dept

#print(no[0],kor[0],eng[0],mat[0],dept[0])

config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True    
}

## DB작업 Table생성 및 레코드 추가
try:
    ## DB연결 객체 및 커서 객체 생성
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    ## table 생성
    sql = """create table if not exists score_tab(
        no int primary key,
        kor int not null,
        eng int not null,
        mat int not null,
        dept int not null
    )"""
    cursor.execute(sql)
    
    cursor.execute("select * from score_tab")
    rows = cursor.fetchall()
    
    if rows:
        print("번호\t국어\t영어\t수학\t반")
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}")
        print("전체 레코드 수 : ",len(rows))
    else:
        print("score.csv 파일을 DB 레코드 추가하기")
        for i in range(len(score)):
            sql = f"insert into score_tab values({no[i]},{kor[i]},{eng[i]},{mat[i]},{dept[i]})"
            cursor.execute(sql)
            conn.commit()
    
except Exception as e:
    print("DB 연동 에러 : ", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()


번호	국어	영어	수학	반
101	88	85	86	101
102	70	90	92	101
103	90	45	55	101
104	55	52	74	102
105	65	85	96	102
106	75	66	41	102
107	62	75	52	103
108	40	51	85	103
109	75	90	63	101
110	50	62	75	102
111	80	55	45	103
112	95	65	85	103
113	82	45	64	101
114	88	78	82	102
115	56	75	75	103
전체 레코드 수 :  15


In [ ]:
## json to table 
#  json은 {키:값} 형식으로 제공하는 파일. 
# 1. json형식의 데이터를 디코딩
# 2. 데이터프레임 형식으로 변경하면 테이블 구조의 컬럼 단위 저장
# #


In [33]:
import pandas as pd
import pymysql
import json

#test = pd.read_json("data/labels.json")
file = open("data/labels.json","r",encoding="utf8")

# 디코딩 :json -> object
lines = json.load(file)
# print(type(lines))          # <class 'list'>
# print(len(lines))           # 30
# print(type(lines[0]))       # <class 'dict'>

## DataFrame 생성
df = pd.DataFrame(lines)
# print(df.info())
# print(df.head())

## DB작업
config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True    
}

try:
    # DB연결 및 SQL 객체 생성
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    sql = """create table if not exists labels(
        id int not null,
        url varchar(150) not null,
        name varchar(50) not null,
        color varchar(50) not null,
        de char(5)        
    )""" 
    cursor.execute(sql)
    
    ## 레코드 조회 후 있는 경우, 출력 | 없으면 추가 
    cursor.execute("select * from labels")
    rows = cursor.fetchall()
    
    if rows:        # 출력
        print("labels 레코드 조회")
        for row in rows:
            print(row)
        print("전체 레코드 수 : ",len(rows))
    else:           # 추가
        for i in range(len(df)):
            uid = df.id[i]     # df.columns
            url = df.url[i]
            name = df['name'][i]
            color = df.color[i]
            de = df.default[i]
            sql = f"insert into labels values({uid},'{url}','{name}','{color}','{de}')"
            cursor.execute(sql)
            conn.commit()
        
except Exception as e:
    print("DB 연동 에러 : ", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

labels 레코드 조회
(76811, 'https://api.github.com/repos/pandas-dev/pandas/labels/Bug', 'Bug', 'e10c02', 'False')
(76812, 'https://api.github.com/repos/pandas-dev/pandas/labels/Enhancement', 'Enhancement', '4E9A06', 'False')
(127681, 'https://api.github.com/repos/pandas-dev/pandas/labels/Refactor', 'Refactor', 'FCE94F', 'False')
(129350, 'https://api.github.com/repos/pandas-dev/pandas/labels/Build', 'Build', '75507B', 'False')
(134699, 'https://api.github.com/repos/pandas-dev/pandas/labels/Docs', 'Docs', '3465A4', 'False')
(233160, 'https://api.github.com/repos/pandas-dev/pandas/labels/Groupby', 'Groupby', '729FCF', 'False')
(2301354, 'https://api.github.com/repos/pandas-dev/pandas/labels/Data%20IO', 'Data IO', '06909A', 'False')
(2413328, 'https://api.github.com/repos/pandas-dev/pandas/labels/Visualization', 'Visualization', '8AE234', 'False')
(2822098, 'https://api.github.com/repos/pandas-dev/pandas/labels/Indexing', 'Indexing', '0b02e1', 'False')
(2822342, 'https://api.github.com/repos/p

In [3]:
### emp.csv 파일을 처리하여 JSON 데이터로 변환 저장하세요... 
## emp.csv -> dict -> JSON 
## 저장된 JSON파일을 불러와서 DB 작업을 하세용!!!!

import pandas as pd
import pymysql
import json

emp = pd.read_csv("data/emp.csv")
emp.No
lst = [ {'No':int(emp.No[i]),'Name':emp.Name[i],'Pay':int(emp.Pay[i])} 
       for i in range(len(emp))]
print(lst)
print(json.dumps(lst,ensure_ascii=False,indent=4))
with open("data/emp.json","w",encoding="utf8") as f:
    json.dump(lst,f,ensure_ascii=False,indent=4)

df = pd.read_json("data/emp.json")

config = {
    'host':'192.168.149.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True    
}

try:
    # DB연결 및 SQL 객체 생성
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    sql = """create table if not exists emp_tab(
        no int not null,
        name varchar(50) not null,
        pay int not null
    )""" 
    cursor.execute(sql)
    
    ## 레코드 조회 후 있는 경우, 출력 | 없으면 추가 
    cursor.execute("select * from emp_tab")
    rows = cursor.fetchall()
    
    if rows:        # 출력
        print("emp_tab 레코드 조회")
        for row in rows:
            print(row)
        print("전체 레코드 수 : ",len(rows))
    else:           # 추가
        for i in range(len(df)):
            no = df.No[i]     # df.columns
            name = df.Name[i]
            pay = df['Pay'][i]
            sql = f"insert into emp_tab values({no},'{name}',{pay})"
            cursor.execute(sql)
            conn.commit()
        
except Exception as e:
    print("DB 연동 에러 : ", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

[{'No': 101, 'Name': '홍길동', 'Pay': 150}, {'No': 102, 'Name': '이순신', 'Pay': 450}, {'No': 103, 'Name': '강감찬', 'Pay': 500}, {'No': 104, 'Name': '유관순', 'Pay': 350}, {'No': 105, 'Name': '김유신', 'Pay': 400}]
[
    {
        "No": 101,
        "Name": "홍길동",
        "Pay": 150
    },
    {
        "No": 102,
        "Name": "이순신",
        "Pay": 450
    },
    {
        "No": 103,
        "Name": "강감찬",
        "Pay": 500
    },
    {
        "No": 104,
        "Name": "유관순",
        "Pay": 350
    },
    {
        "No": 105,
        "Name": "김유신",
        "Pay": 400
    }
]
